# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [14]:
df_curva.head()


,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Considere un swap a 1Y con cupones semestrales de 1.5% (lineal). Si la tasa
cupón cero a 0.5Y es 1.4% (exp), calcule la tasa cero a 1Y.


Pregunta 2:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,010.00 (365 días después).


Pregunta 3:

Una opción sobre un bono cupón cero valorizada con el modelo de Vasicek vale lo
mismo que la misma opción valorizada con el modelo de Hull-White. Comente.


Pregunta 4:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.1%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 5:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 6:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obt

## Respuestas

EXÁMEN ARTURO MORALES VIDELA


### Pregunta 1

In [66]:
tasa_05Y = 0.014
tasa_1Y = math.log(1 + tasa_05Y / 100 * 180 / 360)* 365.0 / 1

print(f' Tasa cero a 1Y: {tasa_1Y}')

 Tasa cero a 1Y: 0.025549105791734767


### Pregunta 2

In [67]:
ICP_0 = 10000
ICP_365 = 10010

TNA = ((ICP_365/ICP_0)-1)*360/365

print(f'Tasa nominal anual a 365 días: {TNA}')


Tasa nominal anual a 365 días: 0.000986301369862905


### Pregunta 3

 Falso, porque el modelo de Vasicek tiene dificultades para calcular la curva cupón cero , mientras que el modelo de Hull White
 no tiene dichas dificultades, de hecho, el modelo de Hull-White se considera una extensión del modelo de Vasicek.
 Al tener distintas maneras de calcular la curva, los factores de descuento no serán los mismos valores ni tampoco las
 tasas esperadas, por lo cual los payoff descontados pueden ser distintos para ambos modelos, lo que significa que los 
 precios de las opciones sobre un bono cero cupón pueden diferir.
 
 Otra diferencia entre ambos modelos es que el modelo de Hull White permite adoptar la medida Forward para valorizar opciones,
 lo cual permite descontar payoff simplemente a la tasa de mercado simulada de la fecha de expiración de la opción.



### Pregunta 4

In [68]:
P_opcion = hw.zcb_call_put(
        hw.CallPut.CALL,
        0.991,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    )

print(f'Precio opción Call a 1Y sobre bono cero cupón a 2Y:  {"{:.8f}".format(P_opcion)}')

Precio opción Call a 1Y sobre bono cero cupón a 2Y:  0.00856130


### Pregunta 5

La principal diferencia entre plantear una simulación de Montecarlo con una medida forward contra una medida libre de 
 riesgo radica en que, en el caso de la medida forward, debemos simular N trayectoria de tasas, luego con la tasa del 
 período final se calcula un payoff con el precio Strike, y ese payoff se descuenta a la tasa del período de expiración
 en el paso final de los "número de steps". Al hacer esto, podemos obtener un total de N payoffs descontados, a los cuales
 se les calcula el promedio y obtenemos finalmente el precio de la opción simulado por MC con N trayectorias. 
 Por ejemplo, si tenemos 264 pasos para una opción a 1 año, simulamos las tasas, luego con las tasas del paso
 264 calculamos el payoff, calculamos el factor de dscto al paso 264 y lo descontamos. Se hace esto N veces y se calcula
 el promedio de todos los payoff descontados.

 En el caso de la medida libre de riesgo, lo que debemos hacer es simular N trayectorias de factores de descuento 
 hasta llegar a la expiración de la opción, con la diferencia de que ahora será necesario componer en el tiempo 
 todos los factores obtenidos en los saltos de tiempo. Esto se logra multiplicando los factores de descuento del paso
 1 hasta el paso paso "final-1 ", es decir, si hacemos 264 pasos, componemos los factores de descuento hasta el 
 paso 263, luego con esos factores compuestos, se obtiene el factor al cual descontamos los payoff y además la tasa
 que se compara con el strike. Esto lo tenemos un total de N veces, luego, promediamos los N payoff descontados y obtenemos
 el precio simulado de la opción.

### Pregunta 6

In [69]:
r_ = curva(0.0001)
dt = 1 / 264
dt_gamma_r_ = dt * gamma * r_
sigma_sqdt = sigma * math.sqrt(dt)
result = [(0, r_),]
gamma_dt = gamma * dt
    
r = dt_gamma_r_ + (1 - gamma_dt) * r_ + sigma_sqdt * (-.5797) # Discretización de Euler


print(f'Un paso de simulación de Vasicek: {r}')

Un paso de simulación de Vasicek: 0.0006278208743051043


### Pregunta 7

In [70]:
#Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
#forward entre 382 y 554 días.
t1 = 382
t2 = 554
r1 = curva(t1)
r2 = curva(t2)
df1 = math.exp(-r1*t1/365)
df2 = math.exp(-r2*t2/365)
tasa_fwd = (df1/df2-1)*365/(t2-t1) #tasa forward
print(f'Tasa forward entre 382 y 554 días: {tasa_fwd}')

Tasa forward entre 382 y 554 días: 0.0005101914539272275
